## selected_kma.csv 만들기

In [1]:
# 데이터프레임이 출력하는 모든 행과 모든 열 출력
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
# 데이터프레임이 출력하는 행과 열 옵션 원래대로 되돌리기
pd.options.display.max_rows = 60
pd.options.display.max_columns = 20
# (Numpy) array 출력 시 생략하지 않고 출력
np.set_printoptions(threshold=np.inf)
# (Numpy) array 원래 옵션
np.set_printoptions(threshold=1000)

In [2]:
import pandas as pd
import numpy as np
import datetime

# {한글 지역명: [STN(발전소 지점별 번호)]}
Location_STN = {
    '강원': [90, 93, 95, 100, 101, 104, 105, 106, 114, 121, 211, 212, 216, 217], 
    '경기': [98, 99, 119, 202, 203], 
    '경남': [155, 162, 192, 253, 255, 257, 263, 264, 284, 285, 288, 289, 294, 295], 
    '경북': [115, 130, 136, 137, 138, 271, 272, 273, 276, 277, 278, 279, 281, 283], 
    '광주': [156], 
    '대구': [143], 
    '대전': [133], 
    '부산': [159, 296], 
    '서울': [108], 
    '세종': [239], 
    '울산': [152], 
    '인천': [102, 112, 201], 
    '전남': [165, 168, 169, 170, 174, 252, 258, 259, 260, 261, 262, 266, 268], 
    '전북': [140, 146, 172, 243, 244, 245, 247, 248, 251, 254], 
    '제주': [184, 185, 188, 189], 
    '충남': [129, 177, 232, 235, 236, 238], 
    '충북': [127, 131, 135, 181, 221, 226]
}

# STN_Location = {STN(발전소 지점별 번호) : '지역'} ex. {90 : '강원', 98 : '경기'}
STN_Location={}
for key in Location_STN.keys():
    for value in Location_STN[key]:
        STN_Location[value]=key

df=pd.read_csv('kma.csv')
df.insert(1,'Location',[STN_Location[val] for val in df['STN']])
# df['YYMMDDHHMI'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y%m%d%H%M%S') # datetime으로 타입 변경(불러오면 항상 데이터 타입 변경해줘야 적용이 됨)

df_ = df.loc[:, ['Location','YYMMDDHHMI', 'STN', 'PA', 'TA', 'HM', 'PV', 'RN', 'RN.1', 'RN.2', 'RN.3', 'SD', 'SD.1', 'SD.2', 'CA', 'CA.1', 'CH', 'VS', 'SS', 'SI', 'TS']]
df_.to_csv('selected_kma.csv',index=False)

## 데이터 전처리

In [518]:
import pandas as pd
import numpy as np
import datetime
import json

In [461]:
df=pd.read_csv('selected_kma.csv')
df['YYMMDDHHMI'] = pd.to_datetime(df['YYMMDDHHMI'], format='%Y-%m-%d %H:%M:%S')
print(df[df['YYMMDDHHMI']>datetime.datetime(2023,6,1,0,0)])
df=df.rename(columns={'Location':'LOC', 'YYMMDDHHMI':'TM', 'RN.1':'RN_DAY', 'RN.2':'RN_JUN', 'RN.3':'RN_INT', 'SD' : 'SD_HR3', 'SD.1': 'SD_DAY', 'SD.2':'SD_TOT', 'CA':'CA_TOT', 'CA.1':'CA_MID', 'CH':'CH_MIN'})
df = df.drop(columns=['RN_INT'])

Empty DataFrame
Columns: [Location, YYMMDDHHMI, STN, PA, TA, HM, PV, RN, RN.1, RN.2, RN.3, SD, SD.1, SD.2, CA, CA.1, CH, VS, SS, SI, TS]
Index: []

[0 rows x 21 columns]


In [462]:
# 지역별로 데이터프레임 나누기
# 지역별로 결측치 집어넣기
local_dic= {'강원': None, '경기': None, '경남': None, '경북': None, '광주': None, '대구': None, '대전': None, '부산': None, '서울': None, '세종': None, '울산': None, '인천': None, '전남': None, '전북': None, '제주': None, '충남': None, '충북': None}

for location in local_dic.keys():
    local_dic[location]=df[df['LOC']==location]

for local in local_dic.values():
    local.loc[local['PA']<0.0,'PA']=np.mean(local[local['PA']>0.0]['PA'])
    local.loc[local['TA']<-30.0,'TA'] = np.mean(local[local['TA']>-30.0]['TA'])
    local.loc[local['HM']<0.0,'HM'] = np.mean(local[local['HM']>0.0]['HM'])
    local.loc[local['PV']<0.0,'PV'] = np.mean(local[local['PV']>0.0]['PV'])
    
    local.loc[local['RN']<0.0,'RN'] = 0.0
    local.loc[local['RN_DAY']<0.0,'RN_DAY'] = 0.0
    local.loc[local['RN_JUN']<0.0,'RN_JUN'] = 0.0
    
    local.loc[local['SD_HR3']<0.0,'SD_HR3'] = 0.0
    local.loc[local['SD_TOT']<0.0,'SD_TOT'] = 0.0
    local.loc[local['SD_DAY']<0.0,'SD_DAY'] = 0.0
    
    local.loc[local['CA_TOT']<0,'CA_TOT'] = int(np.mean(local[local['CA_TOT']>0]['CA_TOT']))
    local.loc[local['CA_MID']<0,'CA_MID'] = int(np.mean(local[local['CA_MID']>0]['CA_MID']))
    local.loc[local['CH_MIN']<0,'CH_MIN'] = int(np.mean(local[local['CH_MIN']>0]['CH_MIN']))
    local.loc[local['VS']<0,'VS'] = int(np.mean(local[local['VS']>0]['VS']))
    local.loc[local['SS']<0.0,'SS'] = 0.0
    local.loc[local['SI']<0.0,'SI'] = 0.0

In [522]:
# for 지역 돌리기:

#     temp_df = None
#     for 시간 돌리기:
#         if 시간있을때: # for i in alltime:
#             df1 = pd.Series(data={'LOC':'강원', 'TM':datetime.datetime(2020,1,1,0,0)})
#             df2 = np.mean(temp.loc[temp['TM']==datetime.datetime(2020,1,1,0,0), 'PA':], axis=0)
#             temp = pd.concat([df1,df2])
#             temp_df = pd.concat(temp_df,temp)
#         else:
#             그 시간 nan 채우기
#         ['TM']j=[nan,nan]
#     지역데이터프레임 = temp_df

C:\Users\user\AppData\Local\Temp\ipykernel_22620\1437131952.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df=pd.concat([temp_df, temp_tm], axis=0, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_22620\1437131952.py:29: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df=pd.concat([temp_df, temp_tm], axis=0, ignore_index=True)


In [542]:
# 총 시간별 데이터수 29928 = 1247일 * 24

temp_all = {'강원': None, '경기': None, '경남': None, '경북': None, '광주': None, '대구': None, '대전': None, '부산': None, '서울': None, '세종': None, '울산': None, '인천': None, '전남': None, '전북': None, '제주': None, '충남': None, '충북': None}

for k in temp_all.keys():
    temp_LOC = local_dic[k].copy()
    temp_df = pd.DataFrame(
        columns=['LOC', 'TM', 'PA', 'TA', 'HM', 'PV', 'RN', 'RN_DAY', 'RN_JUN', 'SD_HR3', 'SD_DAY', 'SD_TOT', 'CA_TOT', 'CA_MID', 'CH_MIN', 'VS', 'SS', 'SI','TS']
    )
    
    for 시간 in alltime:
        df1 = pd.DataFrame(data=[{'LOC': k, 'TM': 시간}])
        
        if 시간 in temp_LOC['TM'].unique():
            df2 = pd.DataFrame(data=[np.mean(temp_LOC.loc[temp_LOC['TM'] == 시간, 'PA':], axis=0)])
            temp_tm = pd.concat([df1, df2], axis=1)
        else:
            temp_tm = df1
        
        temp_df = pd.concat([temp_df, temp_tm], axis=0, ignore_index=True)
    
    # 'TM' 열을 문자열로 변환하여 딕셔너리에 저장
    temp_df['TM'] = temp_df['TM'].astype(str)
    temp_all[k] = temp_df.to_dict(orient='records')

# temp_all.json 저장
with open('temp_all.json', 'w') as file:
    json.dump(temp_all, file)

C:\Users\user\AppData\Local\Temp\ipykernel_22620\305810174.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.concat([temp_df, temp_tm], axis=0, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_22620\305810174.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  temp_df = pd.concat([temp_df, temp_tm], axis=0, ignore_index=True)
C:\Users\user\AppData\Local\Temp\ipykernel_22620\305810174.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 

In [543]:
with open('temp_all.json', 'r') as file:
    all=json.load(file)